In [ ]:
!pip install pandas transformers datasets faiss-gpu-cu11==1.10.0 torch sentence-transformers accelerate bitsandbytes numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Preparacion del ambiente

In [ ]:
import torch
import json
import numpy as np
import pandas as pd
from datasets import Dataset
from huggingface_hub import login
from google.colab import userdata
from sentence_transformers import SentenceTransformer, util
import requests
import json

# Verificar si hay GPU disponible y usarla
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Usando dispositivo: {device}")

Usando dispositivo: cpu


Cargar documentos

In [ ]:
file_path = '/content/drive/MyDrive/UniAndes/MAIA-202411/3. modelos-avanzados-para-el-procesamiento-de-lenguaje-natural/W7/maia-pln-2025/pubmed_500K.json'


In [ ]:
def load_json_to_dataset(file_path):
  try:
    with open(file_path, 'r') as f:
      data = json.load(f)
  except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
    return None

  dataset = Dataset.from_dict(data)
  return dataset

pubmed_dataset = load_json_to_dataset(file_path)
if pubmed_dataset:
  print(pubmed_dataset[0])

{'id': 'pubmed23n0001_0', 'title': "[Biochemical studies on camomile components/III. In vitro studies about the antipeptic activity of (--)-alpha-bisabolol (author's transl)].", 'content': '(--)-alpha-Bisabolol has a primary antipeptic action depending on dosage, which is not caused by an alteration of the pH-value. The proteolytic activity of pepsin is reduced by 50 percent through addition of bisabolol in the ratio of 1/0.5. The antipeptic action of bisabolol only occurs in case of direct contact. In case of a previous contact with the substrate, the inhibiting effect is lost.', 'contents': "[Biochemical studies on camomile components/III. In vitro studies about the antipeptic activity of (--)-alpha-bisabolol (author's transl)]. (--)-alpha-Bisabolol has a primary antipeptic action depending on dosage, which is not caused by an alteration of the pH-value. The proteolytic activity of pepsin is reduced by 50 percent through addition of bisabolol in the ratio of 1/0.5. The antipeptic act

In [ ]:
import pandas as pd

if pubmed_dataset:
  # Convert the dataset to a pandas DataFrame for easier exploration
  df = pd.DataFrame(pubmed_dataset)

  # Display basic information about the DataFrame
  print(df.info())

  # Display the first few rows of the DataFrame
  print(df.head())

  # Summary statistics for numerical columns
  print(df.describe())

  # Check for missing values
  print(df.isnull().sum())

  # Example of exploring specific columns:
  # Assuming there's a column named 'contents'
  if 'contents' in df.columns:
      print("\nExample text entries:")
      print(df['contents'].head(10))  # Print the first 10 text entries

      # Calculate text lengths
      df['text_length'] = df['contents'].str.len()
      print("\nText length statistics:")
      print(df['text_length'].describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        500000 non-null  object
 1   title     500000 non-null  object
 2   content   500000 non-null  object
 3   contents  500000 non-null  object
 4   PMID      500000 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 19.1+ MB
None
                id                                              title  \
0  pubmed23n0001_0  [Biochemical studies on camomile components/II...   
1  pubmed23n0001_1  [Demonstration of tumor inhibiting properties ...   
2  pubmed23n0001_2  Effect of etafenone on total and regional myoc...   
3  pubmed23n0001_3  Influence of a new virostatic compound on the ...   
4  pubmed23n0001_4  Pharmacological properties of new neuroleptic ...   

                                             content  \
0  (--)-alpha-Bisabolol has a primary antipeptic ...   
1  A repor

In [ ]:
# Extraer los textos y los IDs
texts_list = pubmed_dataset['contents']
doc_ids_list = pubmed_dataset['PMID']
doc_ids_list = [str(pmid) for pmid in doc_ids_list]

#Filtrado de textos vacios o None
filtered_texts = []
filtered_doc_ids = []

for text, doc_id in zip(texts_list, doc_ids_list):
    if text and text.strip():  # Verifica que no sea None y que no esté vacío después de quitar espacios
        filtered_texts.append(text)
        filtered_doc_ids.append(doc_id)
    else:
        print(f"Advertencia: Documento con ID {doc_id} tiene campo 'contents' vacío o nulo. Será omitido.")


texts = filtered_texts
doc_ids = filtered_doc_ids

if not texts:
    print("No se extrajeron textos válidos del Dataset. Verifica el contenido de la columna 'contents'.")
    # Salir o manejar el error apropiadamente
    exit()

print(f"\nNúmero de textos válidos extraídos: {len(texts)}")
print(f"Número de IDs de documentos válidos extraídos: {len(doc_ids)}")

if texts: # Solo mostrar si hay textos
    print(f"Ejemplo del primer texto extraído: '{texts[0][:200]}...'") # Muestra los primeros 200 caracteres
    print(f"Ejemplo del primer ID extraído: {doc_ids[0]}")




Número de textos válidos extraídos: 500000
Número de IDs de documentos válidos extraídos: 500000
Ejemplo del primer texto extraído: '[Biochemical studies on camomile components/III. In vitro studies about the antipeptic activity of (--)-alpha-bisabolol (author's transl)]. (--)-alpha-Bisabolol has a primary antipeptic action dependi...'
Ejemplo del primer ID extraído: 21


Cargamos el modelo microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext para usarlo como Encoder

In [ ]:
HF_TOKEN = userdata.get('HF_TOKEN')

if HF_TOKEN is None:
    raise ValueError("HF_TOKEN environment variable not set.")

login(token=HF_TOKEN)

In [ ]:
# Load ENCODER model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

encoder_model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext"

#Load the model
encoder = SentenceTransformer(encoder_model_name, device=device)

embedding_dim = encoder.get_sentence_embedding_dimension()
print(f"Dimensión del embedding: {embedding_dim}")


config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/226k [00:00<?, ?B/s]

Dimensión del embedding: 768


Generar Embeddings para los Documentos

In [ ]:
print("Generando embeddings para los documentos ...")

corpus_embeddings = encoder.encode(texts, show_progress_bar=True, batch_size=128, convert_to_numpy=True)

print(f"Forma de los embeddings del corpus: {corpus_embeddings.shape}")

# Guardar los embeddings y los IDs para uso futuro
np.save("pubmed_embeddings_pubmedbert.npy", corpus_embeddings)
with open("pubmed_doc_ids_pubmedbert.json", "w") as f:
    json.dump(doc_ids, f)

print("Embeddings y IDs guardados.")

Generando embeddings para los documentos (esto puede tardar)...


Batches:   0%|          | 0/3907 [00:00<?, ?it/s]

Forma de los embeddings del corpus: (500000, 768)
Embeddings y IDs guardados.


In [ ]:
import numpy as np
import datetime

# Save embeddings and IDs with timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
embeddings_filename = f"/content/drive/MyDrive/UniAndes/MAIA-202411/3. modelos-avanzados-para-el-procesamiento-de-lenguaje-natural/W7/maia-pln-2025/encoder/pubmed_embeddings_pubmedbert_{timestamp}.npy"
ids_filename = f"/content/drive/MyDrive/UniAndes/MAIA-202411/3. modelos-avanzados-para-el-procesamiento-de-lenguaje-natural/W7/maia-pln-2025/encoder/pubmed_doc_ids_pubmedbert_{timestamp}.json"

np.save(embeddings_filename, corpus_embeddings)
with open(ids_filename, "w") as f:
    json.dump(doc_ids, f)

print(f"Embeddings saved to: {embeddings_filename}")
print(f"IDs saved to: {ids_filename}")

Embeddings saved to: /content/drive/MyDrive/UniAndes/MAIA-202411/3. modelos-avanzados-para-el-procesamiento-de-lenguaje-natural/W7/maia-pln-2025/encoder/pubmed_embeddings_pubmedbert_20250516_020423.npy
IDs saved to: /content/drive/MyDrive/UniAndes/MAIA-202411/3. modelos-avanzados-para-el-procesamiento-de-lenguaje-natural/W7/maia-pln-2025/encoder/pubmed_doc_ids_pubmedbert_20250516_020423.json


Construir el indice faiss

In [ ]:
print("Construyendo índice FAISS...")

dimension = corpus_embeddings.shape[1]

index = faiss.IndexFlatL2(dimension)
index.add(corpus_embeddings)
print(f"Total de vectores en el índice: {index.ntotal}")

# Guardar el índice FAISS
faiss_index_path = "pubmed_faiss_pubmedbert.index"
faiss.write_index(index, faiss_index_path)
print(f"Índice FAISS guardado en: {faiss_index_path}")

Construyendo índice FAISS...
Total de vectores en el índice: 500000
Índice FAISS guardado en: pubmed_faiss_pubmedbert.index


In [ ]:
# Guardar el índice FAISS con timestamp
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
faiss_index_path_timestamped = f"/content/drive/MyDrive/UniAndes/MAIA-202411/3. modelos-avanzados-para-el-procesamiento-de-lenguaje-natural/W7/maia-pln-2025/encoder/pubmed_faiss_pubmedbert_{timestamp}.index"
faiss.write_index(index, faiss_index_path_timestamped)
print(f"Índice FAISS guardado con timestamp en: {faiss_index_path_timestamped}")

Índice FAISS guardado con timestamp en: /content/drive/MyDrive/UniAndes/MAIA-202411/3. modelos-avanzados-para-el-procesamiento-de-lenguaje-natural/W7/maia-pln-2025/encoder/pubmed_faiss_pubmedbert_20250516_020433.index
